# 0. set up

Simply clone the distil-whisper repo!

In [2]:
!git clone -b main-dev https://github.com/eustlb/distil-whisper.git

fatal: destination path 'distil-whisper' already exists and is not an empty directory.


# 1. run pseudo labelling

We run here the pseudo labelling step, where audio are concatenated to 30 seconds samples and pseudo labelled using whisper-large-v3. To do so, the targeted dataset is streamed from the Hugging Face hub and processed on the fly. For this reason, it is necessary for your dataset to be beforehand on the hub. The result will be saved to disk and pushed to the hub under your username (to avoid reprocessing it!). 

Please set the below parameters to the values corresponding to your Hugging Face Hub dataset. Refer to the given example using Common Voice 17 spanish config!

## parameters

In [8]:
dataset_name = "mozilla-foundation/common_voice_17_0"
dataset_config_name = "es" 
audio_column_name = "audio"
text_column_name = "sentence"
id_column_name = "client_id"
language = "es"
dataset_split_name = "train" # remains unchanged
model_name_or_path = "openai/whisper-large-v3" # remains unchanged

## launch pseudo labelling!

In [ ]:
import os

# Build the command string
command = f"""
chmod +x ./run-scripts/run_pseudo_labelling.sh
./run-scripts/run_pseudo_labelling.sh "{model_name_or_path}" "{dataset_name}" "{dataset_config_name}" "{dataset_split_name}" "{audio_column_name}" "{text_column_name}" "{language}" "{id_column_name}"
"""

# Execute the command
os.system(command)

## clean temporary data

In [3]:
!rm -rf /data/tmp

# 2. training

## 2.1 create student model

In [ ]:
!python /data/distil-whisper/training/create_student_model.py \
  --teacher_checkpoint "distil-whisper/distil-large-v3" \
  --encoder_layers 32 \
  --decoder_layers 2 \
  --save_dir "/data/student-model"

## 2.2 train the model

### parameters

In [ ]:
# to be changed to your specific case
language = "es"

# training set 
train_dataset_name = "eustlb/common_voice_17_0_es_pseudo_labelled"
train_split_name = "train"
text_column_name = "sentence"

# validation set
eval_dataset_name = "eustlb/common_voice_17_0_es_pseudo_labelled"
eval_split_name = "validation"
eval_text_column_name = "sentence"

output_dir = "/data/distil-large-v3-es"

In [ ]:
# training parameters, should remain unchanged
model_name_or_path = "/data/distil-large-v3-es"
max_steps = 80000 # optimization steps
warmup_steps = 500
learning_rate = 0.0001
timestamp_probability = 0.5
condition_on_prev_probability = 0.2
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
dataloader_num_workers = 4
wer_threshold = 20

### launch training!

In [ ]:
import os

# Build the command string
command = f"""
chmod +x ./run-scripts/run_training.sh
./run-scripts/run_training.sh "{model_name_or_path}" "{train_dataset_name}" "{train_split_name}" "{text_column_name}" "{eval_dataset_name}" "{eval_split_name}" "{eval_text_column_name}" "{warmup_steps}" "{learning_rate}" "{timestamp_probability}" "{condition_on_prev_probability}" "{language}" "{max_steps}" "{wer_threshold}" "{per_device_train_batch_size}" "{per_device_eval_batch_size}" "{dataloader_num_workers}" "{output_dir}"
"""

# Execute the command
os.system(command)

# 3. Evaluate the trained model